# Imports & Settings

In [1]:
import pandas as pd
import numpy as np

# Display options

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Load Data & Initial Snapshot

In [4]:
df = pd.read_csv('cleaned_combined_listings.csv')
print(f"Initial shape: {df.shape}")
df.head()

Initial shape: (5297, 6)


,title,price,location,area,bathrooms,bedrooms
0,ADV905*4BHK Villa for rent in Madinat Illam in...,750.0,"Qurum, Muscat•",300.0,4.0,4.0
1,ADV906**4BHK Villa in a comples in shatti qurum,1900.0,"Qurum, Muscat•",300.0,4.0,4.0
2,ADC507*** Office Space in Azaiba – 440 sqm for...,2310.0,"Azaiba, Muscat•",440.0,NaN,NaN
3,*ADV705** 3+1 BHK Villa for Rent in Bousher –A...,650.0,"Bosher, Muscat•",350.0,4.0,3.0
4,4 BR + Maid’s Room Spacious Well-Designed Vill...,750.0,"Azaiba, Muscat•",439.0,5.0,4.0


# Overview of Missing Values

In [5]:
missing_pct = df.isna().mean().sort_values(ascending=False)
missing_pct.to_frame(name='pct_missing')

,pct_missing
bedrooms,0.218048
bathrooms,0.201812
price,0.179724
area,0.141401
location,0.018879
title,0.000000


#  Drop or Impute High‑Missing Columns
# Drop columns with >50% missing

In [6]:
high_missing = missing_pct[missing_pct > 0.5].index.tolist()
df.drop(columns=high_missing, inplace=True)

# Impute numerical with median, categorical with mode

In [7]:
num_cols = df.select_dtypes(include='number').columns
cat_cols = df.select_dtypes(include='object').columns

for c in num_cols:
    if df[c].isna().any():
        df[c].fillna(df[c].median(), inplace=True)

for c in cat_cols:
    if df[c].isna().any():
        df[c].fillna(df[c].mode()[0], inplace=True)

C:\Users\DuaaHilal\AppData\Local\Temp\ipykernel_16740\937108172.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)
C:\Users\DuaaHilal\AppData\Local\Temp\ipykernel_16740\937108172.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

# Remove Exact & Near‑Duplicate Rows

In [9]:
dup_count = df.duplicated().sum()
print(f"Exact duplicates to drop: {dup_count}")
df.drop_duplicates(inplace=True)

key = ['title', 'price', 'location']
near_dup = df.duplicated(subset=key).sum()
print(f"Near‑duplicates to drop: {near_dup}")
df.drop_duplicates(subset=key, inplace=True)

Exact duplicates to drop: 1
Near‑duplicates to drop: 74


# Standardize Text Columns

In [15]:
for c in ['title', 'description', 'location']:
    if c in df.columns:
        df[c] = df[c].str.strip().str.lower()

if 'location' in df.columns:
    df['location'] = df['location'].replace({
        'muscat city': 'muscat',
        'muscat governorate': 'muscat'
    })

C:\Users\DuaaHilal\AppData\Local\Temp\ipykernel_16740\2696287885.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = df[c].str.strip().str.lower()
C:\Users\DuaaHilal\AppData\Local\Temp\ipykernel_16740\2696287885.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location'] = df['location'].replace({


# Handle Outliers in Numeric Features

In [16]:
# Only process specified columns if they exist
cols_to_check = [c for c in ['price', 'area_sqft', 'bedrooms'] if c in df.columns]
for col in cols_to_check:
    before = df.shape[0]
    df = remove_outliers(col)
    after = df.shape[0]
    print(f"{col}: dropped {before-after} outliers")

price: dropped 144 outliers
bedrooms: dropped 58 outliers


# Feature‑Specific Cleaning / Parsing

In [18]:
if 'posted_date' in df.columns:
    df['posted_date'] = pd.to_datetime(df['posted_date'], errors='coerce')
    median_date = df['posted_date'].median()
    df['posted_date'].fillna(median_date, inplace=True)

if 'features' in df.columns:
    df['num_features'] = df['features'].str.count(',') + 1

# Consistency Checks & Final Sanity

In [19]:
assert df.isna().sum().sum() == 0, "There are still missing values!"
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,4294,3936,room for rent,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,4294.0,NaN,NaN,NaN,307.719953,208.089197,0.0,120.0,340.0,400.0,890.0
location,4294,119,"salalah, dhofar•",502,NaN,NaN,NaN,NaN,NaN,NaN,NaN
area,4294.0,NaN,NaN,NaN,3097.436122,45592.298056,1.0,65.0,120.0,200.0,1000000.0
bathrooms,4294.0,NaN,NaN,NaN,2.38612,1.437721,0.0,1.0,2.0,3.0,10.0
bedrooms,4294.0,NaN,NaN,NaN,2.173032,1.188739,1.0,1.0,2.0,3.0,6.0


#  Drop or flag zero‑price (and zero‑bath) listings

In [20]:
df = df[df['price'] > 0]
df.loc[df['bathrooms'] == 0, 'bathrooms'] = df['bathrooms'].median()

#  Cap extreme outliers

In [21]:
for col in ['price','area','bedrooms']:
    low, high = df[col].quantile([0.01,0.99])
    df = df[df[col].between(low, high)]

# Clean text

In [23]:
df['location'] = (
    df['location']
      .str.replace('•','', regex=False)
      .str.strip()
      .str.lower()
)

# Log‑transform skewed numerics for modeling

In [25]:
for col in ['price','area']:
    df[f'log_{col}'] = np.log1p(df[col])

# Consistency Checks & Final Sanity

In [26]:
assert df.isna().sum().sum() == 0, "There are still missing values!"
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,4211,3858,room for rent,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,4211.0,NaN,NaN,NaN,309.796599,207.132761,10.0,125.0,340.0,400.0,850.0
location,4211,115,"salalah, dhofar",494,NaN,NaN,NaN,NaN,NaN,NaN,NaN
area,4211.0,NaN,NaN,NaN,252.016198,757.01286,10.0,65.0,115.0,200.0,11111.0
bathrooms,4211.0,NaN,NaN,NaN,2.401092,1.436507,1.0,1.0,2.0,3.0,10.0
bedrooms,4211.0,NaN,NaN,NaN,2.177393,1.195646,1.0,1.0,2.0,3.0,6.0
log_price,4211.0,NaN,NaN,NaN,5.429462,0.905372,2.397895,4.836282,5.831882,5.993961,6.746412
log_area,4211.0,NaN,NaN,NaN,4.70418,1.127904,2.397895,4.189655,4.75359,5.303305,9.315781


#  Save Final Clean CSV

In [27]:
df.to_csv('cleaned_combined_listings_second_cleaned.csv', index=False)
print("Saved")

Saved
